In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import re
import pandas as pd
import matplotlib.pylab as plt

from image_analysis.read_files import get_paths, change_paths
from image_analysis.fit import plot, roi, regionprops
from image_analysis.preprocess import mask
from image_analysis.align_surface import *

from tqdm import tqdm

In [3]:
pixel_size = 2

In [4]:
folder_pretreated = r'example_pretreated_images/'

In [5]:
img_paths = get_paths(folder_pretreated, exclude='.lif_', suffix='.npy')
img_paths = change_paths(img_paths, '', 'pixel')
img_paths = change_paths(img_paths, '', 'keys')

names = [os.path.basename(file).split('.npy')[0] for file in img_paths]

In [6]:
df = pd.DataFrame(names, columns=['pretreated_file_name'])
df['hcr'] = [ int(name.split('_')[0].split('hcr')[1]) for name in df['pretreated_file_name'] ]

In [7]:
df

,pretreated_file_name,hcr
0,hcr21_2_5cm_7dpa_noif,21
1,hcr21_2_5cm_6dpa_p_hist,21


In [8]:
for i, file_name in tqdm( enumerate( df['pretreated_file_name'].values) ):

    print(i, file_name)
    
    file = os.path.join( folder_pretreated, file_name) + '.npy'
    
    keys = np.load( file.split('.npy')[0] + '_keys.npy' )
    image = dict(zip(
        keys,
        np.load( file )
    ))

    # align
    surface = make_surf(image['tissue_mask_with_epi'])
    surface = surface_crop_mask(surface, image['tissue_mask_with_epi'], 0.01)

    regionprops_3d = regionprops.regionprops_3d( image, 1, 1, 1, pixel_size)
    image          = regionprops_3d.rotate_surface_pca_to_ez(surface, 2 )

    # update
    np.save( file, np.array(list(image.values())) )

0it [00:00, ?it/s]

0 hcr21_2_5cm_7dpa_noif
do not need to rotate around x
rotation around y:
0 -2.2081507697602945 0
dictionary, with 3d data
angle beetween ey and surface PCA, 2: -2


1it [00:13, 13.98s/it]

1 hcr21_2_5cm_6dpa_p_hist
rotation around x:
6.545299626072064 0 0
dictionary, with 3d data
rotation around y:
0 7.468080113772495 0
dictionary, with 3d data
strange rotation
angle beetween ey and surface PCA, 2: -7


2it [00:30, 15.22s/it]
